In [4]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [30]:
# Give a url for respective year
response = requests.get("https://fbref.com/en/comps/22/1759/schedule/2018-Major-League-Soccer-Fixtures")

In [31]:
soup = BeautifulSoup(response.content, features= "html.parser")

In [32]:
boxScores = []
for i in range(len(soup.find_all("tr"))):
    try:
        boxScores.append(soup.find_all("tr")[i].find_all("a")[-1].get('href'))
    except IndexError:
        pass

In [33]:
len(boxScores)

816

In [35]:
def refinedInfo(lst,split_el):
    name = ['day','venue time', 'Venue', 'Attendance']
    dct = dict(zip(name,[0]*len(name)))
    for n in lst:
        t = n.split(split_el) 
        if 'venue time' in n:
            dct['venue time'] = n.split(', ')[-1].replace(' (venue time) ','')
            dct['day'] = ','.join(n.split(',')[:2])
        elif 'Attendance' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
        elif 'Venue' in n:
            elm_1 = t[0]
            elm_2 = t[1]
            dct[elm_1] = elm_2
    
    return list(dct.values())

In [36]:
col_name = ['Team_1','Team_1_score','Team_2','Team_2_score','Day&Date','Start Time','Stadium','Attendance']
df = pd.DataFrame(columns= col_name)
for i,half_url in enumerate(boxScores):
    url = "https://fbref.com" + half_url
    response_1 = requests.get(url)
    soup_1 = BeautifulSoup(response_1.content, features= "html.parser")
    
    pr = soup_1.find_all('div',attrs= {'class':"scores"})
    
    if pr != []:
        
        if (pr[0].find_all('div',attrs= {'class':"score"})[0].get_text() == '0') and \
        (pr[1].find_all('div',attrs= {'class':"score"})[0].get_text() == '0'):
            try:
                score = [s.find('div',attrs= {'class':"score_pen"}).get_text() for s in pr]
            except AttributeError:
                score = [s.find('div',attrs= {'class':"score"}).get_text() for s in pr]
        else:
            score = [s.find('div',attrs= {'class':"score"}).get_text() for s in pr]

        teams = soup_1.title.get_text().split(' Match Report')[0].split(' vs. ')[:2]
        txt_1 = soup_1.find_all("div", attrs= {'class':"scorebox_meta"})
        imd = [j.get_text() for j in txt_1[0].find_all("div")]
        imd_2 = refinedInfo(imd,': ')
        df.loc[i] = [teams[0]] + [score[0]] + [teams[1]] + [score[1]] +imd_2
    else:
        pass

In [37]:
df.shape

(816, 8)

In [38]:
df

,Team_1,Team_1_score,Team_2,Team_2_score,Day&Date,Start Time,Stadium,Attendance
0,Toronto FC,0,Columbus Crew,2,"Saturday March 3, 2018",13:00,"BMO Field, Toronto, ON","26,633"
1,Houston Dynamo,4,Atlanta United,0,"Saturday March 3, 2018",14:30,"BBVA Compass Stadium, Houston, TX","20,377"
2,Philadelphia Union,2,New England Revolution,0,"Saturday March 3, 2018",19:00,"Talen Energy Stadium, Chester, PA","16,452"
3,FC Dallas,1,Real Salt Lake,1,"Saturday March 3, 2018",19:00,"Toyota Stadium, Frisco, TX","16,116"
4,Orlando City,1,D.C. United,1,"Saturday March 3, 2018",19:30,"Orlando City Stadium, Orlando, FL","25,527"
5,San Jose Earthquakes,3,Minnesota United,2,"Saturday March 3, 2018",19:30,"Avaya Stadium , San José, CA","18,000"
6,Seattle Sounders FC,0,Los Angeles FC,1,"Sunday March 4, 2018",14:00,"CenturyLink Field, Seattle, WA","40,070"
7,Vancouver Whitecaps FC,2,Montreal Impact,1,"Sunday March 4, 2018",15:00,"BC Place Stadium, Vancouver, BC","27,837"
8,Sporting KC,0,New York City FC,2,"Sunday March 4, 2018",18:30,"Children's Mercy Park, Kansas City, MO","20,831"
9,LA Galaxy,2,Portland Timbers,1,"Sunday March 4, 2018",19:00,"StubHub Center, Carson, CA","25,462"


In [ ]:
# df.to_excel("2018.xlsx")